<a target="_blank" href="https://colab.research.google.com/github/PabloOrazi/fed-sentiment-analysis/blob/main/1.1%20-%20Bajada%20de%20Datos%20y%20Confecci%C3%B3n%20del%20Dataframe.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Instalación de paquetes necesarios

Primero instalamos el paquete FedTools para descargar las minutas de la FED. Si está ejecutando este archivo desde su computadora en vez de Google Colab, instale el resto de los requerimientos a través del comando ```pip install -r requirements.txt```.

In [3]:
!pip install FedTools

     -------------------------------------- 109.5/109.5 kB 3.2 MB/s eta 0:00:00


## Descarga de información para el modelo  

El propósito del modelo será pronosticar el siguiente movimiento de tasa de la FED. Para ello, deberemos descargar y construir un conjunto de Features. Nuestro interés será ver si una vez que disponemos de la minuta de la última reunión podemos mejorar con el uso de un modelo de análisis de lenguaje la predicción que descuentan los mercados. 

La predicción que hacen los mercados es observable a través del diferencial entre las tasas de interés respecto a la tasa de referencia. Si la tasa de referencia es 4.50%, pero la tasa a un plazo de 3 meses es 4.75%, el mercado descuenta que habrá una suba, ya que invertir a 3 meses rinde significativamente más que la tasa actual.


### Descarga de tasas de interés  

En primer lugar, obtendremos del sitio FRED (Federal Reserve Economic Data) datos sobre las tasas de interés de referencia y la tasa de 3 meses. Para la tasa de referencia, se usará por un lado la tasa efectiva que se opera (que se la promedia para 5 días para evitar señales falsas por factores de iliquidez transitorios), y por otro la tasa objetivo. Y para la tasa objetivo, se combinarán los datos históricos, cuando se usaba una tasa de referencia, empalmándolos con los datos a partir del momento en que se empezó a trabajar con piso y techo para la tasa de referencia.

In [4]:
import pandas_datareader.data as web
import datetime
import pandas as pd
from FedTools import MonetaryPolicyCommittee
from FedTools import FederalReserveMins

In [11]:
start_date = datetime.datetime(1950, 1, 1)
ff_rate = web.DataReader([
                "DFF",
                'DFEDTAR',
                "DFEDTARL",
                "DFEDTARU"], 'fred', start_date)
ff_rate_mid = ff_rate[['DFEDTAR',
                "DFEDTARL",
                "DFEDTARU"]].mean(axis=1).ffill()
effective_ff_rate = ff_rate["DFF"].ffill().rolling(5).mean().dropna()
effective_ff_rate = ff_rate["DFF"].ffill()

other_short_rates = web.DataReader(['DTB3',], 'fred', start_date).ffill()

### Descarga de minutas de las reuniones de la Reserva Federal
Luego usamos el paquete```FedTools``` para descargar las minutas y comunicados de las reuniones de la Fed. Aplicaremos un modelo de análisis de sentimiento (Hawkish, Neutral, Dovish) a estas minutas en el siguiente notebook.

In [18]:
start_year = 1970
historical_split = 2018
dataset_statements = MonetaryPolicyCommittee(start_year=start_year, historical_split=historical_split).find_statements()
dataset_minutes = FederalReserveMins(historical_split=historical_split).find_minutes()

Constructing links between 1970 and 2024
Extracting the past 227 FOMC Statements.
Retrieving articles.
...................................................................................................................................................................................................................................Constructing links between 1995 and 2024
Extracting Federal Reserve Minutes.
Retrieving articles.
..............................................................................................................................................................................................................................................

### Construcción de un dataframe donde se consolide información de fechas relevantes

Incluimos en el repositorio un archivo llamado *date_minutes.pkl* en el cual incluimos las fechas de las reuniones y las fechas de la publicación de las minutas respectivas, para saber la fecha en que se dispone de esta información. 

Utilizaremos un ajuste de un día hacia delante, para representar un momento en que la información de la minuta habría sido absorbida por los mercados. 

Con la información del análisis de sentimiento de la minuta intentamos ver si obtenemos una información adicional o más objetiva en comparación con el sentimiento de mercado. 

In [45]:
try:
    path_minutes_published='Datos/date_minutes.pkl'
    df_fechas_pub_min=pd.read_pickle(path_minutes_published)
except FileNotFoundError:
    print("Descargando de GitHub")
    path_minutes_published='https://github.com/PabloOrazi/fed-sentiment-analysis/raw/main/Datos/date_minutes.pkl'
    df_fechas_pub_min=pd.read_pickle(path_minutes_published)    
    
df_fechas_pub_min.columns=['Meeting', 'Fecha Minuta']
df_fechas_pub_min['Meeting'] = pd.to_datetime(df_fechas_pub_min['Meeting'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df_fechas_pub_min['Fecha Minuta'] = pd.to_datetime(df_fechas_pub_min['Fecha Minuta'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df_fechas_pub_min.set_index('Meeting', inplace=True)
df_fechas_pub_min.index = pd.to_datetime(df_fechas_pub_min.index)
df_fechas_pub_min

,Fecha Minuta
Meeting,
1995-02-01,1995-03-31
1995-03-28,1995-05-26
1995-05-23,1995-07-07
1995-07-06,1995-08-25
1995-08-22,1995-09-29
...,...
2023-06-14,2023-07-05
2023-07-26,2023-08-16
2023-09-20,2023-10-11


### Consolidación de la información de fechas

Creamos un dataframe donde tenemos la fecha de la reunión, la publicación de la minuta, la fecha de la siguiente reunión, y fechas defasadas para representar un momento en que ya los mercados hubiesen recibido y procesado la minuta y las decisiones de política monetaria hubiesen sido tomadas.

In [38]:
fechas_pub_minutas = pd.merge(dataset_minutes, df_fechas_pub_min, left_index=True, right_index=True)

fechas = pd.DataFrame({
    "Fecha Reunión" : dataset_minutes.index[:],
    "Publicacion Minuta" : fechas_pub_minutas['Fecha Minuta'],})

fechas["Publicacion Minuta"] = pd.to_datetime(fechas["Publicacion Minuta"])
fechas["Fecha Reunión + 1 BD"] = fechas["Fecha Reunión"] + pd.offsets.BDay()
fechas["Publicacion Minuta + 1 BD"] = fechas["Publicacion Minuta"]+ pd.offsets.BDay()
fechas["Fecha Siguiente Reunión"] = list(dataset_minutes.index[1:])+[None]
fechas["Fecha Siguiente Reunión + 1 BD"] = fechas["Fecha Siguiente Reunión"]+ pd.offsets.BDay()
fechas

,Fecha Reunión,Publicacion Minuta,Fecha Reunión + 1 BD,Publicacion Minuta + 1 BD,Fecha Siguiente Reunión,Fecha Siguiente Reunión + 1 BD
1995-02-01,1995-02-01,1995-03-31,1995-02-02,1995-04-03,1995-03-28,1995-03-29
1995-03-28,1995-03-28,1995-05-26,1995-03-29,1995-05-29,1995-05-23,1995-05-24
1995-05-23,1995-05-23,1995-07-07,1995-05-24,1995-07-10,1995-07-06,1995-07-07
1995-07-06,1995-07-06,1995-08-25,1995-07-07,1995-08-28,1995-08-22,1995-08-23
1995-08-22,1995-08-22,1995-09-29,1995-08-23,1995-10-02,1995-09-26,1995-09-27
...,...,...,...,...,...,...
2023-06-14,2023-06-14,2023-07-05,2023-06-15,2023-07-06,2023-07-26,2023-07-27
2023-07-26,2023-07-26,2023-08-16,2023-07-27,2023-08-17,2023-09-20,2023-09-21
2023-09-20,2023-09-20,2023-10-11,2023-09-21,2023-10-12,2023-11-01,2023-11-02
2023-11-01,2023-11-01,2023-11-21,2023-11-02,2023-11-22,2023-12-13,2023-12-14


### Creación del Dataframe de Fechas y Tasas
Incorporamos al dataframe anterior datos de tasas, incluyendo la tasa de referencia objetivo, la tasa promedio observada los últimos 5 días, la tasa de 3 meses y la decisión de política monetaria, que se expresa en cuál es la diferencia entre la tasa en la siguiente reunión respecto a la tasa luego de esta reunión.

In [42]:
tasas = pd.concat(
    (ff_rate_mid,
     effective_ff_rate,
    other_short_rates), axis=1
)
tasas.columns =["Fed Target Rate", "Effective Fed Funds Rate", "Tasa 3 Meses"]

df_fechas = fechas[['Fecha Reunión + 1 BD',
 'Publicacion Minuta + 1 BD',
 'Fecha Siguiente Reunión + 1 BD']]
variacion_tasas = df_fechas.merge(tasas, left_on='Fecha Siguiente Reunión + 1 BD', right_index=True, how="left")["Fed Target Rate"]-df_fechas.merge(tasas, left_on='Fecha Reunión + 1 BD', right_index=True, how="left")["Fed Target Rate"]
df_fechas["Decisión Política Monetaria"] = variacion_tasas
df_fechas = df_fechas.merge(tasas, left_on='Publicacion Minuta + 1 BD', right_index=True, how='left')

df_fechas

C:\Users\pablo\AppData\Local\Temp\ipykernel_14764\2977799455.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fechas["Decisión Política Monetaria"] = variacion_tasas


,Fecha Reunión + 1 BD,Publicacion Minuta + 1 BD,Fecha Siguiente Reunión + 1 BD,Decisión Política Monetaria,Fed Target Rate,Effective Fed Funds Rate,Tasa 3 Meses
1995-02-01,1995-02-02,1995-04-03,1995-03-29,0.00,6.000,6.28,5.70
1995-03-28,1995-03-29,1995-05-29,1995-05-24,0.00,6.000,5.97,5.66
1995-05-23,1995-05-24,1995-07-10,1995-07-07,-0.25,5.750,5.78,5.36
1995-07-06,1995-07-07,1995-08-28,1995-08-23,0.00,5.750,5.72,5.31
1995-08-22,1995-08-23,1995-10-02,1995-09-27,0.00,5.750,5.93,5.26
...,...,...,...,...,...,...,...
2023-06-14,2023-06-15,2023-07-06,2023-07-27,0.25,5.125,5.08,5.23
2023-07-26,2023-07-27,2023-08-17,2023-09-21,0.00,5.375,5.33,5.29
2023-09-20,2023-09-21,2023-10-12,2023-11-02,0.00,5.375,5.33,5.35
2023-11-01,2023-11-02,2023-11-22,2023-12-14,0.00,5.375,5.33,5.27


### Exportación de datos para utilizar en siguientes fases del proceso

In [43]:
import os

outdir = './Datos'
if not os.path.exists(outdir):
    os.mkdir(outdir)

df_fechas.to_pickle("Datos/df_fechas.pkl")
tasas.to_pickle("Datos/tasas.pkl")
dataset_minutes.to_pickle("Datos/dataset_minutes.pkl")
dataset_statements.to_pickle("Datos/dataset_statements.pkl")

### Descarga de datos de Google Colab

Si se ejecutan los datos en Google Colab, habrá que descargar la información para usar en los pasos subsiguientes, en los cuales habrá que subir los datos al Google Colab.

In [44]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive, files
    files.download(r'df_fechas.pkl')
    files.download(r'tasas.pkl')
    files.download(r'dataset_minutes.pkl')
    files.download(r'dataset_statements.pkl')